In [1]:
# This is one option for reading in the config file - but then would have to alter the rest of the script to eliminate config.variable, etc. 
# With this method, just need the variable names from config file and not config.variable_name
# Load the configuration settin
# exec(open(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation\config.py').read())

In [1]:
# Check Current Working Directory
import os
print(os.getcwd())

S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation


In [2]:
# Change working directory to the notebook's directory if needed 
# This ensures the config.py file in the same directory as the notebook can be referenced
os.chdir(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation')

In [3]:
# Module search path: Also ensures that the directory containing 'config.py' is in the Python search path ('sys.path').
import sys
sys.path.append(r'S:\Projects\ABC\y2025\Pro\Draft\ABC_RSR_Calculation_Pro\ABC_RSR_Calculation')

In [4]:
import arcpy
import os
import config

Environments set from config file.
Paths set from config file.
File and variable names set from config file.


In [5]:
# Check if the project folders (final, intermediate) exist, if not, create them
if not os.path.exists(config.project_folder_final):
    os.makedirs(config.project_folder_final)
    
if not os.path.exists(config.project_folder_int):
    os.makedirs(config.project_folder_int)

In [6]:
# Create two geodatabases, final and intermediate, check if the gdb's exist

# Create final gdb
gdb_path_f = os.path.join(config.project_folder_final, f"{config.gdb_final}.gdb")
if arcpy.Exists(gdb_path_f):
    print(f"Geodatabase '{config.gdb_final}.gdb' already exists.")
else: 
    arcpy.CreateFileGDB_management(config.project_folder_final, config.gdb_final)
    print(f"Geodatabase '{config.gdb_final}.gdb' created successfully.")

# Create intermediate gdb
gdb_path_i = os.path.join(config.project_folder_int, f"{config.gdb_int}.gdb")
if arcpy.Exists(gdb_path_i):
    print(f"Geodatabase '{config.gdb_int}.gdb' already exists.")
else:
    arcpy.CreateFileGDB_management(config.project_folder_int, config.gdb_int)
    print(f"Geodatabase '{config.gdb_int}.gdb' created successfully.")

workspace_final = os.path.join(config.project_folder_final, f"{config.gdb_final}.gdb")
workspace_int = os.path.join(config.project_folder_int, f"{config.gdb_int}.gdb")

Geodatabase 'ABC_final.gdb' already exists.
Geodatabase 'ABC_intermediate.gdb' already exists.


In [7]:
# read in the ABC_HabitatMap Attribute EXPORT csv to the intermediate gdb

arcpy.env.workspace = workspace_int

arcpy.TableToTable_conversion(config.input_csv, os.path.join(config.project_folder_int, f"{config.gdb_int}.gdb"), config.output_table)

<Result 'S:\\Projects\\ABC\\y2025\\Data\\Intermediate\\intermediateABC_y2025\\ABC_intermediate.gdb\\ABCrsr_table'>

In [8]:
# Create a new empty table to store the results

arcpy.env.workspace = workspace_final

# Check if the table already exists

if arcpy.Exists(config.ABC_Analysis_Table):
    overwrite = input(f"Table '{config.ABC_Analysis_Table}' already exists. Do you want to overwrite it? Type 'yes' to proceed: ")
    if overwrite.lower() != 'yes':
        print("Exiting script, The table was not overwritten.")
        exit()
    else:
        arcpy.Delete_management(config.ABC_Analysis_Table)
        print(f"Table '{config.ABC_Analysis_Table}' deleted.")
        
# Create new empty table
    
arcpy.CreateTable_management(arcpy.env.workspace, config.ABC_Analysis_Table)
fields_to_fetch = ["Subtype", "HabitatCod", "Subtype_Count"]
print(fields_to_fetch)

for field in fields_to_fetch:
    # Determine the field type for each field
    field_type = None
    
    # Get the field type from the output table
    for f in arcpy.ListFields(config.output_table):
        if f.name == field:
            field_type = f.type
            break
            
    # If the field is "Primary_Count", set its type to "DOUBLE"
    if field == "Subtype_Count":
        field_type = "DOUBLE"
        
    # Add the field to the new table
    arcpy.AddField_management(config.ABC_Analysis_Table, field, field_type)
    
# OLD CODE
#for field in fields_to_fetch:
    # Add fields to the new table based on fields_to_fetch
    #field_type = [f.type for f in arcpy.ListFields(config.output_table) if f.name == field][0]
    #arcpy.AddField_management(config.ABC_RSR_Analysis_Table, field, field_type)

['Subtype', 'HabitatCod', 'Subtype_Count']


In [9]:
# Open a search cursor to read data from the input table

with arcpy.da.SearchCursor(config.output_table, fields_to_fetch) as cursor:
    # Open an insert cursor to insert data into the new table
    with arcpy.da.InsertCursor(config.ABC_Analysis_Table, fields_to_fetch) as insert_cursor:
        for row in cursor:
            print(row)
            # Insert each row from the search cursor into the new table
            insert_cursor.insertRow(row)

('Subarctic Tundra Woodland', 10100, 266097841)
('Boreal Conifer', 10200, 2512646772)
('Northern Spruce-Fir Forest', 10300, 128362418)
('Rocky Mountain Spruce-Fir Forest', 10302, 175996634)
('General Nearctic Montane Mixed Conifer Forest', 10400, 67516481)
('Rocky Mountain Mixed Conifer Forest', 10404, 294241766)
('Sierra Nevada - Columbia Plateau Mixed Conifer Forest', 10405, 91267832)
('Ponderosa Pine Forest', 10500, 102973924)
('General Eastern Pine Savanna', 10600, 172617)
('Shortleaf Pine', 10606, 30528944)
('Longleaf Pine', 10607, 62724996)
('Northeastern Pine Barrens', 10608, 3973766)
('Appalachian Coniferous Forest', 10700, 25127775)
('Nearctic High-elevation Pine Woodland', 10800, 1864924)
('Jack Pine Forest', 10900, 12370426)
('Lodgepole Pine Forest', 11000, 64598189)
('Nearctic Temperate Rainforest', 11100, 290764747)
('Pinyon-Juniper Woodland', 11200, 187136115)
('Madrean Pine-Oak Woodland', 11300, 7884325)
('Chihuahuan Desert', 21400, 159600251)
('Sonoran Desert', 21500, 1

In [10]:
# Add in a series of fields to record the output in

table = config.ABC_Analysis_Table

field_type = "DOUBLE"
precision = 38
scale = 15

field_names = ["Subtype_sqkm", "Subtype_hectares", "Subtype_RSR", "GAP_1_2_Count", "GAP_1_2_Area", "GAP_1_2_3_Count", "GAP_1_2_3_Area", "Protected_Area_1_2", "Protected_Area_1_2_3", "Protected_Area_US_GAP3"]

for field_name in field_names:
    arcpy.AddField_management(
        in_table=table, 
        field_name=field_name, 
        field_type=field_type, 
        field_precision=precision, 
        field_scale=scale
    )
    
print("New table created successfully.")    

New table created successfully.
